Họ tên: Võ Thành Nam

MSSV: 19120301

# HW3: Các loại bộ nhớ trong CUDA

Với các GPU tương đối mới thì để biên dịch chỉ cần dùng câu lệnh: \
`nvcc tên-file.cu -o tên-file-chạy`

Nhưng trên Colab mình thường lấy được GPU khá cũ là Tesla K80 với compute capability (phiên bản phần cứng) là 3.7; để biên dịch đúng với GPU khá cũ này thì bạn cần dùng câu lệnh: \
`nvcc -arch=sm_37 tên-file.cu -o tên-file-chạy` \
Trong đó, 37 chính là compute capability của GPU Tesla K80.

Để phòng trường hợp khi làm bài bạn lấy được GPU có compute capability x.x nhưng khi chấm bài Thầy lại lấy được GPU có compute capability khác x.x, dưới đây mình sẽ có đoạn code Python để tự động lấy 2 con số ứng với compute capability của GPU và lưu vào 2 biến `major` và `minor`:


In [1]:
from numba import cuda
major, minor = cuda.get_current_device().compute_capability
print(f'GPU compute capability: {major}.{minor}')

GPU compute capability: 7.5


Một khi đã chạy đoạn code Python ở trên, để biên dịch thì bạn sẽ dùng câu lệnh: \
`nvcc -arch=sm_{major}{minor} tên-file.cu -o tên-file-chạy`

Dưới đây, khi làm bài thì bạn có thể tùy ý thêm/xóa cell. Đừng xóa mấy cell có chữ của Thầy là được.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# You should change this
%cd /content/drive/MyDrive/ParallelProgramming/HW3

/content/drive/MyDrive/ParallelProgramming/HW3


In [4]:
!nvcc -arch=sm_{major}{minor} HW3.cu -o HW3
!./HW3 in.pnm out.pnm 

**********GPU info**********
Name: Tesla T4
Compute capability: 7.5
Num SMs: 40
Max num threads per SM: 1024
Max num warps per SM: 32
GMEM: 15843721216 bytes
CMEM: 65536 bytes
L2 cache: 4194304 bytes
SMEM / one SM: 65536 bytes
****************************

Image size (width x height): 512 x 512

Kernel 1, block size 32x32, grid size 16x16
Kernel time: 0.349120 ms
Error: 0.000703

Kernel 2, block size 32x32, grid size 16x16
Kernel time: 0.338624 ms
Error: 0.000703

Kernel 3, block size 32x32, grid size 16x16
Kernel time: 0.258016 ms
Error: 0.000703


In [5]:
!./HW3 in.pnm out.pnm 16 16

**********GPU info**********
Name: Tesla T4
Compute capability: 7.5
Num SMs: 40
Max num threads per SM: 1024
Max num warps per SM: 32
GMEM: 15843721216 bytes
CMEM: 65536 bytes
L2 cache: 4194304 bytes
SMEM / one SM: 65536 bytes
****************************

Image size (width x height): 512 x 512

Kernel 1, block size 16x16, grid size 32x32
Kernel time: 0.460960 ms
Error: 0.000703

Kernel 2, block size 16x16, grid size 32x32
Kernel time: 0.313344 ms
Error: 0.000703

Kernel 3, block size 16x16, grid size 32x32
Kernel time: 0.239616 ms
Error: 0.000703


In [6]:
!./HW3 in.pnm out.pnm 8 8

**********GPU info**********
Name: Tesla T4
Compute capability: 7.5
Num SMs: 40
Max num threads per SM: 1024
Max num warps per SM: 32
GMEM: 15843721216 bytes
CMEM: 65536 bytes
L2 cache: 4194304 bytes
SMEM / one SM: 65536 bytes
****************************

Image size (width x height): 512 x 512

Kernel 1, block size 8x8, grid size 64x64
Kernel time: 0.804320 ms
Error: 0.000703

Kernel 2, block size 8x8, grid size 64x64
Kernel time: 0.506336 ms
Error: 0.000703

Kernel 3, block size 8x8, grid size 64x64
Kernel time: 0.447968 ms
Error: 0.000703


Giải thích kết quả:

- Việc dùng SMEM nhanh hơn so với không dùng vì SMEM có tốc độ truy xuất cao hơn so với truy xuất qua GMEM, hơn nữa việc dùng SMEM cũng sẽ hạn chế số lần truy xuất GMEM.
- Việc dùng CMEM nhanh hơn so với không dùng vì dữ liệu của filter không thay đổi trong suốt quá trình thực thi. CMEM có bộ nhớ const cache có độ trễ thấp. Trong warp các thread sẽ lấy dữ liệu lần lượt các phần tử từ cùng 1 filter, mỗi phần tử chỉ đọc 1 lần duy nhất và được broadcast cho toàn bộ các thread trong warp, từ đó làm giảm đáng kể thời gian thực thi.